<a href="https://colab.research.google.com/github/fwitschel/WIMA/blob/main/notebooks/Expert_RAG_IRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Execute this code only if in colab
if 'COLAB_GPU' in os.environ:
  print("Executing in Colab!")
  # Cloning GitHub repository
  !git clone https://github.com/fwitschel/WIMA.git
  %cd WIMA


Executing in Colab!
Cloning into 'WIMA'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 43 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 1.64 MiB | 3.69 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/WIMA



First, we will install required dependencies.

In [2]:
!pip install langchain langchain-community pypdf sentence_transformers faiss-cpu langchain-anthropic groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
import pandas as pd
df = pd.read_csv('/content/WIMA/data/IRF_Data_small.csv', sep=";")
df.head()

,person,dc.title,dc.subtitle
0,"Grunder, Hans-Ulrich","Der Bildungsforschung die Daten, der Allgemein...",NaN
1,"Kirnbauer, Martin",NaN,NaN
2,"Hell, Benedikt",Entwicklung eines Interviewleitfadens für die ...,NaN
3,"Herrmann, Anne",Entwicklung eines Interviewleitfadens für die ...,NaN
4,"Labudde, Peter",Didaktische Leitfragen für die Lehrerinnen- un...,Beiträge zur Lehrerbildung (BZL)


### Parsing the documents
First, we will load the csv document and parse it using Pandas.

In [4]:
from langchain_core.documents import Document
docs = []
for index, row in df.iterrows():
    person = row['person']
    title = row['dc.title']
    document = Document(
        page_content= "Author: " + str(person) + ", Title: " + str(title),
        metadata={"source": "IRF"}
    )
    docs.append(document)

print(docs[0])

page_content='Author: Grunder, Hans-Ulrich, Title: Der Bildungsforschung die Daten, der Allgemeinen Pädagogik die skeptischen RückFragen? Bildungsforschung und Allgemeine Pädagogik' metadata={'source': 'IRF'}


In [5]:
print(docs[0].page_content[0:250])

Author: Grunder, Hans-Ulrich, Title: Der Bildungsforschung die Daten, der Allgemeinen Pädagogik die skeptischen RückFragen? Bildungsforschung und Allgemeine Pädagogik


Once we have the text from the document, we have to split it into smaller chunks. We can use LangChain's available splitters, like CharacterTextSplitter in this case:

In [6]:
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunks = splitter.split_documents(docs)
print(chunks[0])

page_content='Author: Grunder, Hans-Ulrich, Title: Der Bildungsforschung die Daten, der Allgemeinen Pädagogik die skeptischen RückFragen? Bildungsforschung und Allgemeine Pädagogik' metadata={'source': 'IRF'}


We will be using BGE-small, an opensource embeddings model. We will download it from HuggingFace Hub and run it on all chunks to calculate their vector representations.

In [7]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

chunk_texts = list(map(lambda d: d.page_content, chunks))
embeddings = bge_embeddings.embed_documents(chunk_texts)
print(embeddings[0])

/tmp/ipython-input-1782701561.py:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  bge_embeddings = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.0302888173609972, 0.028282642364501953, -0.02332819439470768, -0.0003163748770020902, 0.023050880059599876, -0.007087355013936758, -0.002219259738922119, 0.038638100028038025, -0.02305730991065502, -0.0008127346518449485, 0.0270692165941, -0.04541735351085663, -0.00552099896594882, 0.012048673816025257, 0.021680952981114388, 0.009791522286832333, -0.0031466290820389986, 0.014194335788488388, -0.008890214376151562, 0.027788061648607254, 0.05815925821661949, -0.03392795845866203, 0.0010215151123702526, -0.03422490134835243, 0.009577189572155476, 0.033019132912158966, 0.0018786747241392732, -0.01642192341387272, -0.012252326123416424, -0.19010557234287262, -0.018888987600803375, -0.032895732671022415, 0.04011224955320358, -0.022394562140107155, 0.0011473004706203938, -0.024210166186094284, 0.033582594245672226, 0.01834167167544365, -0.022503500804305077, 0.027211390435695648, -0.005444013979285955, 0.02085844613611698, 0.027021845802664757, -0.016583075746893883, -0.002433497225865721

Once we have the vector representations for all chunks, we can create an in-memory vector database and store all vectors in it. For this example, we will be using a FAISS database.

In [8]:
from langchain_community.vectorstores import FAISS

text_embedding_pairs = zip(chunk_texts, embeddings)
db = FAISS.from_embeddings(text_embedding_pairs, bge_embeddings)

The database is now set up. Now, we will be taking queries from the user on this information. In the case of expert search, we expect the user to ask for an area of expertise. Then, we retrieve the top k most similar chunks to that query.

In [14]:
topk = 42
query = "knowledge management"

contexts = db.similarity_search(query, k=topk)

for i in range(topk):
  print(contexts[i].page_content)


Author: Karagiannis, Dimitris (external), Title: Knowledge management concepts and tools: The PROMOTE project.
Author: Woitsch, Robert (external), Title: Knowledge management concepts and tools: The PROMOTE project.
Author: Telesko, Rainer, Title: Knowledge management concepts and tools: The PROMOTE project.
Author: Liebig, Brigitte, Title: 'Tacit knowledge' und Management: Ein wissenssoziologischer Beitrag zur qualitativen Organisationskulturforschung
Author: Hinkelmann, Knut, Title: WM2009: 5th Conference on Professional Knowledge Management.
Author: Nentwig-Gesemann, Iris, Title: 'Tacit knowledge' und Management: Ein wissenssoziologischer Beitrag zur qualitativen Organisationskulturforschung
Author: Bohnsack, Ralf, Title: 'Tacit knowledge' und Management: Ein wissenssoziologischer Beitrag zur qualitativen Organisationskulturforschung
Author: Hinkelmann, Knut, Title: Towards an Integrated Approach to Assess the Potential of an Enterprise to Mature Knowledge, 5th Conference Profession

After retrieving the relevant context, we build a prompt using this information and the user's original query. We will use a Llama model via the Huggingface API:



> This example uses Huggingface API to call the model. In order for it to work, remember to set the Secret Variable "HUGGINGFACE_API_KEY" to your own Huggingface API Key, or change the model to any of your choice.



In [15]:
from groq import Groq
def llm(groq_client, prompt):
  chat_completion = groq_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
  )

  return chat_completion.choices[0].message.content

In [16]:
from google.colab import userdata
groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY')
)

In [19]:
context = '\n\n'.join(list(map(lambda c: c.page_content, contexts)))
prompt = f"""You are an assistant that helps users to find people with a certain expertise, based on their publications.
You need to analyse the publications and check who has most relevant publications in the given field! The following context contains several relevant publications;
for each publication, the author is provided first: {context}. Please only return internal experts!
The field that we are interested in is {query}"""
response = llm(groq_client, prompt)
print(response)

After analyzing the provided publications, I have identified the top internal experts in the field of Knowledge Management. Here are the results:

1. **Hinkelmann, Knut** (14 publications): Knut Hinkelmann is the most prominent internal expert in Knowledge Management, with 14 relevant publications. His research focuses on various aspects of Knowledge Management, including process execution, agile processes, service-oriented knowledge and learning architectures, and document management.

Other notable internal experts in Knowledge Management are:

2. **Telesko, Rainer** (2 publications): Rainer Telesko has co-authored publications on Knowledge Management concepts and tools, as well as assessing the potential of an enterprise to mature knowledge.
3. **Liebig, Brigitte** (1 publication): Brigitte Liebig has co-authored a publication on "Tacit knowledge" and management, which is a relevant topic in Knowledge Management.
4. **Nentwig-Gesemann, Iris** (1 publication): Iris Nentwig-Gesemann h